In [5]:
## download the french vectors

#! curl -o ./wiki.multi.fr.vec https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import pickle

from gensim.models import KeyedVectors

In [7]:
en_vec = KeyedVectors.load_word2vec_format('../../Data/GoogleNews-vectors-negative300.bin', binary=True)
fr_vec = KeyedVectors.load_word2vec_format('../../Data/wiki.multi.fr.vec')

In [8]:
def get_dict(filename):
    '''
        returns a dictionary representation of the data
    '''
    data = pd.read_csv(filename, sep=' ')
    
    output = {}
    
    for i in range(len(data)):
        en = data.loc[i][0]
        fr = data.loc[i][1]
        
        output[en] = fr
        
    return output

In [9]:
en_fr_train = get_dict('../data/machine_translation/en-fr.train.txt')
en_fr_test = get_dict('../data/machine_translation/en-fr.test.txt')

print(f'The train dataset length : {len(en_fr_train)}')
print(f'The test dataset length : {len(en_fr_test)}')

The train dataset length : 5000
The test dataset length : 1500


## Creating a subset of embedding

In [10]:
en_set = set(en_vec.vocab)
fr_set = set(fr_vec.vocab)

en_embeddings_subset = {}
fr_embeddings_subset = {}

french_words = set(en_fr_train.values())

for en_word in en_fr_train.keys():
    fr_word = en_fr_train[en_word]
    if fr_word in fr_set and en_word in en_set:
        en_embeddings_subset[en_word] = en_vec[en_word]
        fr_embeddings_subset[fr_word] = fr_vec[fr_word]
        
for en_word in en_fr_test.keys():
    fr_word = en_fr_test[en_word]
    if fr_word in fr_set and en_word in en_set:
        en_embeddings_subset[en_word] = en_vec[en_word]
        fr_embeddings_subset[fr_word] = fr_vec[fr_word]

pickle.dump(en_embeddings_subset, open('en_embeddings.p','wb'))
pickle.dump(fr_embeddings_subset, open('fr_embeddings.p','wb'))

## Load the embedding subset

In [11]:
en_embeddings_subset = pickle.load(open("en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("fr_embeddings.p", "rb"))

In [12]:
en_embeddings_subset['the'].size

300

## Generate embeddings and transform matrics

In [13]:
def get_matrices(en_fr, french_vecs, english_vecs):
    '''
        Generate X and Y vectors for english-french
    '''
    
    X_l = []
    y_l = []
    
    english_set = english_vecs.keys()
    french_set = french_vecs.keys()
    
    french_words = set(en_fr.values())
    
    for eng, fr in en_fr.items():
        
        if fr in french_set and eng in english_set:
            
            eng_vec = english_vecs[eng]
            fr_vec = french_vecs[fr]
        
            X_l.append(eng_vec)
            y_l.append(fr_vec)
            
    X = np.vstack(X_l)
    y = np.vstack(y_l)
    
    return X,y

In [14]:
X_train, y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [15]:
X_train.shape

(4932, 300)

## Translation as linear transformation of embeddings

Step 1: Computing the loss
The loss function will be squared Frobenoius norm of the difference between matrix and its approximation, divided by the number of training examples $m$.
Its formula is: $$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$
where $a_{i j}$ is value in $i$th row and $j$th column of the matrix $\mathbf{XR}-\mathbf{Y}$.

Instructions: 
- complete the compute_loss() function
- Compute the approximation of Y by matrix multiplying X and R
- Compute difference XR - Y
- Compute the squared Frobenius norm of the difference and divide it by $m$.

In [16]:
def compute_loss(X, Y, R):
    '''
        Computing Loss using Frobenoius norm.
        Inputs: 
            X: a matrix of dimension (m,n) where the columns are the English embeddings.
            Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
            R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
        Outputs:
            L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    '''
    m = X.shape[0]
    
    diff = np.dot(X,R) - Y
    
    diff_sqrd = diff ** 2
    
    sum_diff_sqrd = np.sum(diff_sqrd)
    
    loss = sum_diff_sqrd / m
    
    return loss

In [17]:
def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        g: a matrix of dimension (n,n) - gradient of the loss function L for given X, Y and R.
    '''
    # m is the number of rows in X
    m = X.shape[0]

    # gradient is X^T(XR - Y) * 2/m
    gradient = np.dot(X.transpose(),np.dot(X,R)-Y)*(2/m)
    
    return gradient

In [18]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
    '''
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    '''
    np.random.seed(129)

    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
            
        # use the function that you defined to compute the gradient
        gradient = compute_gradient(X,Y,R)

        # update R by subtracting the learning rate times gradient
        R -=  learning_rate * gradient
        
    return R

In [19]:
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = align_embeddings(X, Y)

loss at iteration 0 is: 3.7242
loss at iteration 25 is: 3.6283
loss at iteration 50 is: 3.5350
loss at iteration 75 is: 3.4442


In [21]:
R_train = align_embeddings(X_train, y_train, train_steps=400, learning_rate=0.8)

loss at iteration 0 is: 963.0146
loss at iteration 25 is: 97.8292
loss at iteration 50 is: 26.8329
loss at iteration 75 is: 9.7893
loss at iteration 100 is: 4.3776
loss at iteration 125 is: 2.3281
loss at iteration 150 is: 1.4480
loss at iteration 175 is: 1.0338
loss at iteration 200 is: 0.8251
loss at iteration 225 is: 0.7145
loss at iteration 250 is: 0.6534
loss at iteration 275 is: 0.6185
loss at iteration 300 is: 0.5981
loss at iteration 325 is: 0.5858
loss at iteration 350 is: 0.5782
loss at iteration 375 is: 0.5735


## Nearest Neighbour and Test

In [24]:
def cosine_similarity(A,B):
    '''
        Returns the cosine similarity between vectors A and B
    '''
    
    d = np.dot(A,B)
    norm_a = np.sqrt(np.dot(A,A))
    norm_b = np.sqrt(np.dot(B,B))
    
    cos = d / (norm_a * norm_b)
    
    return cos

In [26]:
def nearest_neighbor(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    similarity_l = []

    # for each candidate vector...
    for row in candidates:
        # get the cosine similarity
        cos_similarity = cosine_similarity(v,row)

        # append the similarity to the list
        similarity_l.append(cos_similarity)
        
    # sort the similarity list and get the indices of the sorted list
    sorted_ids = np.argsort(similarity_l)

    # get the indices of the k most similar candidate vectors
    k_idx = sorted_ids[-k:]
    
    return k_idx

In [27]:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 3)])

[[9 9 9]
 [1 0 5]
 [2 0 1]]


In [28]:
def test_vocabulary(X, Y, R):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''
    
    # The prediction is X times R
    pred = np.dot(X,R)

    # initialize the number correct to zero
    num_correct = 0

    # loop through each row in pred (each transformed embedding)
    for i in range(len(pred)):
        # get the index of the nearest neighbor of pred at row 'i'; also pass in the candidates in Y
        pred_idx = nearest_neighbor(pred[i],Y)

        # if the index of the nearest neighbor equals the row of i... \
        if pred_idx == i:
            # increment the number correct by 1.
            num_correct += 1

    # accuracy is the number correct divided by the number of rows in 'pred' (also number of rows in X)
    accuracy = num_correct / len(pred)
    

    return accuracy

In [29]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [30]:
acc = test_vocabulary(X_val, Y_val, R_train)
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.557
